# Notebook for model training

### Training Flow

Load Training Data:
- Load set of input data and label files
- Prefilter positive set only to rule out heavily-masked patches
- Create train/test set
- Define augmentation parameters

Train Model:
- Define model architecture
- Compile model
- Train and evaluate model
- Save model

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import date
import os
import pickle
import sys

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm.notebook import tqdm

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from scripts import viz_tools

## Open Data

In [ ]:
resolution = 64

data_files = [
    'random_land_clay_centroids_2023-01-01_2023-12-31_patch_arrays.pkl', 
    'active_surface_mines_2023-01-01_2023-12-31_patch_arrays.pkl',
    'v0.1_negatives_2023-01-01_2023-12-31_patch_arrays.pkl'
    ]
label_files = [
    'random_land_clay_centroids_2023-01-01_2023-12-31_patch_array_labels.pkl', 
    'active_surface_mines_2023-01-01_2023-12-31_patch_array_labels.pkl', 
    'v0.1_negatives_2023-01-01_2023-12-31_patch_array_labels.pkl'
    ]

patches = []
labels = []

data_dir = os.path.join('..', 'data', 'training_data', f"{resolution}_px")

for data, label in tqdm(zip(data_files, label_files), total=len(data_files)):
    with open(os.path.join(data_dir, data), 'rb') as f:
        data = pickle.load(f)
        for elem in data:
            #patch = dl_utils.pad_patch(elem, resolution)
            patches.append(elem)
    with open(os.path.join(data_dir, label), 'rb') as f:
        label = pickle.load(f)
        labels = np.concatenate((labels, label))

patches = np.array(patches)
positive_patches = patches[labels == 1]
negative_patches = patches[labels == 0]

print(len(patches), "samples loaded")
print(sum(labels == 1), "positive samples")
print(sum(labels == 0), "negative samples")

In [ ]:
num_samples = 64
indices = np.random.randint(0, len(patches), num_samples)
viz_tools.plot_image_grid(patches[indices], labels=[int(label) for label in labels[indices]])

In [ ]:
def filter_white(data, threshold, return_rejects=True):
    """Filter out patches that are mostly white.
    
    Args:
        data (np.array): Array of patches.
        threshold (float): Threshold for white pixels.
        return_rejects (bool): If True, return the rejected patches.
        
    Returns:
        np.array: Filtered patches.
        np.array: Rejected patches (if return_rejects is True).
    """
    rejects = []
    for i, patch in enumerate(data):
        if np.median(patch[:,:,1]) > threshold:
            rejects.append(i)
    if return_rejects:
        return np.delete(data, rejects, axis=0), data[rejects]
    else:
        return np.delete(data, rejects, axis=0)

In [ ]:
# Filter positive pixels that are masked beyond a threshold. Don't want to give positive examples of cloud-masked patches
filtered_positives, positive_rejects = filter_white(positive_patches, threshold= 2_500, return_rejects=True)
if len(positive_rejects) > 0:
    num_samples = 16**2
    if len(positive_rejects) < num_samples:
        num_samples = len(positive_rejects)
    indices = np.random.randint(0, len(positive_rejects), num_samples)
    viz_tools.plot_numpy_grid(positive_rejects[indices,:,:,3:0:-1] / 3000)
    plt.title("Positive Masked Rejects")
    plt.show()

num_samples = 25 ** 2
indices = np.random.randint(0, len(filtered_positives), num_samples)
fig = viz_tools.plot_numpy_grid(filtered_positives[indices,:,:,3:0:-1] / 3000)
plt.title('Positive Filtered Samples')
plt.show()

In [ ]:
filtered_negatives, negative_rejects = filter_white(positive_patches, threshold= 2_500, return_rejects=True)
if len(negative_rejects) > 0:
    num_samples = 16**2
    if len(positive_rejects) < num_samples:
        num_samples = len(positive_rejects)
    indices = np.random.randint(0, len(negative_rejects), num_samples)
    viz_tools.plot_numpy_grid(negative_rejects[indices,:,:,3:0:-1] / 3000)
    plt.title("Negative Mask Rejects")
    plt.show()

num_samples = 25 ** 2
indices = np.random.randint(0, len(filtered_negatives), num_samples)
fig = viz_tools.plot_numpy_grid(filtered_negatives[indices,:,:,3:0:-1] / 3000)
plt.title('Negative Filtered Samples')
plt.show()

## Prepare Data for Training

In [ ]:
#for RGBIR, x = normalize(np.copy(images[:,:,:,[1,2,3,8]]))
x = patches
y = labels
x, y = shuffle(x, y, random_state=33)

## Data Augmentation
Augmentation is important to help the model train. These are parameters that generally worked, but there is certainly room for improvement.

Plot a set of images to give an example of augmented images

In [ ]:
augmentation_parameters = {
    'featurewise_center': False,
    'rotation_range': 360,
    'width_shift_range': [0.9, 1.1],
    'height_shift_range': [0.9, 1.1],
    'shear_range': 10,
    'zoom_range': [0.9, 1.1],
    'vertical_flip': True,
    'horizontal_flip': True,
    # Fill options: "constant", "nearest", "reflect" or "wrap"
    'fill_mode': 'reflect'
}

datagen = ImageDataGenerator(**augmentation_parameters)


plt.figure(figsize=(12,12), facecolor=(1,1,1), dpi=150)
aug_img, aug_labels = datagen.flow(x, y, batch_size=64).next()
viz_tools.plot_image_grid(aug_img, labels=[int(l) for l in aug_labels], norm=True);


In [ ]:
x_norm = np.clip(np.array(x.astype("float32") / 10000), 0, 1)

## Train Ensemble

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.30)
print(f"Min Value:\t\t {np.min(x_train):.3f}")
print(f"Max Value:\t\t {np.max(x_train):.3f}")
print(f"Median Value:\t\t {np.median(x_train):.3f}")
print("Num Train:\t\t", len(x_train))
print("Num Test:\t\t", len(x_test))
print(f"Percent Negative Train:\t {100 * sum(y_train == 0.0) / len(y_train):.1f}")
print(f"Percent Negative Test:\t {100 * sum(y_test == 0.0) / len(y_test):.1f}")

num_classes = 2
input_shape = x_train.shape[1:]
print("Input Shape:", input_shape)

model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3), padding='same', activation="relu"),
        layers.Conv2D(32, kernel_size=(3), padding='same', activation="relu"),
        layers.Conv2D(32, kernel_size=(3), padding='same', activation="relu"),
        layers.MaxPooling2D(pool_size=(2)),
        layers.Conv2D(32, kernel_size=(3), padding='same', activation="relu"),
        layers.Conv2D(32, kernel_size=(3), padding='same', activation="relu"),
        layers.Conv2D(32, kernel_size=(3), padding='same', activation="relu"),
        layers.MaxPooling2D(pool_size=(2)),
        layers.Conv2D(32, kernel_size=(3), padding='same', activation="relu"),
        layers.Conv2D(32, kernel_size=(3), padding='same', activation="relu"),
        layers.Conv2D(32, kernel_size=(3), padding='same', activation="relu"),
        layers.MaxPooling2D(pool_size=(3)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')])
model.summary()
model.compile(
optimizer=keras.optimizers.Adam(3e-4), 
loss=keras.losses.BinaryCrossentropy(from_logits=False), 
metrics=[keras.metrics.BinaryAccuracy(name="acc")])

train_accuracy = []
test_accuracy = []

batch_size = 32
epochs = 64
# add a weighted loss
class_weight = {0: 1, 1: 1}

model.fit(datagen.flow(x_train, y_train),
        batch_size=batch_size, 
        epochs=epochs, 
        validation_data = (x_test, y_test),
        verbose = 1,
        shuffle=True,
        class_weight = class_weight
        )
train_accuracy += model.history.history['acc']
test_accuracy += model.history.history['val_acc']


plt.figure(figsize=(8,5), dpi=100, facecolor=(1,1,1))
plt.plot([i for i in range(1, len(train_accuracy) + 1)], train_accuracy, label='Train Acc')
plt.plot([i for i in range(1, len(train_accuracy) + 1)], test_accuracy, c='r', label='Val Acc')
percent_negative = (sum(y_train == 0.0) / len(y_train))
plt.plot([1, len(train_accuracy) + 1], [percent_negative, percent_negative], '--', c='gray', label='Baseline')
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Network Train and Val Accuracy')
plt.show()

In [ ]:
threshold = 0.8
print("Test Set Metrics:")
preds = model.predict(x_test)
print(classification_report(y_test, preds > threshold, 
                        target_names=['No Mine', 'Mine']))

version_number = f'0.2'
current_date = date.today()
model_name = f"{resolution}px_v{version_number}_{current_date.isoformat()}"

#assert not os.path.exists('../models/' + model_name + '.h5'), f"Model of name {model_name} already exists"

with open('../models/' + model_name + '_config.txt', 'w') as f:
        f.write('Input Data:\n')
        [f.write('\t' + file + '\n') for file in data_files]
        f.write('\n\nAugmentation Parameters:\n')
        for k, v in zip(augmentation_parameters.keys(), augmentation_parameters.values()):
                f.write(f"\t{k}: {v}\n")
        f.write(f"\nBatch Size: {batch_size}")
        f.write(f"\nTraining Epochs: {len(train_accuracy)}")
        f.write(f'\n\nClassification Report at {threshold}\n')
        f.write(classification_report(y_test, model.predict(x_test) > threshold, 
                                target_names=['No Mine', 'Mine']))
                

model.save(f'../models/{model_name}.h5')

# Evaluate Model Performance Characteristics

Find the threshold that maximizes performance on the test set. Note that while this may be the optimum performance on the test set, it does not account for the fact that false positives are functionally worse than false negatives.

In [ ]:
test_model = model

val_images = x_test
val_labels = y_test

test_preds = test_model.predict(val_images)
thresh = []
score = []
for threshold in range(2, 100, 2):
    threshold /= 100
    thresh.append(threshold)
    test_labels = [np.argmax(y) for y in val_labels]
    test_out = [pred > threshold for pred in test_preds]
    score.append(1 - (np.sum(np.array(test_labels) != np.array(test_out)[:,0]) / len(test_labels)))
    #print(np.sum(np.array(test_labels) != np.array(test_preds)), "of", len(test_labels), "test set predictions incorrect")
plt.plot(thresh, score)
plt.ylabel('Success Rate')
plt.xlabel('Threshold')
plt.title(f"Optimal Threshold: {thresh[np.argmax(score)]:.2f}")
plt.show()
optimal_threshold = thresh[np.argmax(score)]

Plot images that the model classifies incorrectly. Can be useful to evaluate model bias.

In [ ]:
threshold = 0.8
test_model = model
val_images = x_test
val_labels = y_test
test_labels = val_labels
test_preds = test_model.predict(val_images)
for index, (label, pred, img) in enumerate(zip(test_labels, test_preds, val_images)):
    pred = pred[0]
    if pred < threshold:
        binary_pred = 0
    else:
        binary_pred = 1
    if label != binary_pred:
        rgb = (img[:,:,3:0:-1] * 10000 / 3000)
        fig = plt.figure(figsize=(2,2), facecolor=(1,1,1), dpi=150)
        plt.imshow(np.clip(rgb, 0, 1))
        plt.title(f"label: {label} - pred: {pred:.2f}")
        plt.axis('off')
        plt.show()